# Commit Reveal Strategy
This commit reveal strategy aims to tackle unwanted weight copying behaviour bittensor. This notebook will guide you through how to run diagnostic in your SN to determine a correct parameter to set for your SN. 

## Background
Validators are encouraged to do validation work to increase competitveness in a subnet. Through reaching consensus, validators would be rewarded with dividend for their contribution in the SN. Yet, some validators choose to reach consensus through direct copying weights that was produced by other validators which hurts the decentralization characteristic of bittensor.

Commit reveal was designed such that there will be offset in time when the weights are generated by the validators and the concensus are calculated. Thus, the weights that weight copier set would always be at least `commit_reveal_weight_interval` later than the original weight. The idea is that `commit_reveal_weight_interval` should be long enough such that when the copier does the copy, there would be enough change in the network, so that the weight to copy would already be irrelevant.

Here it illustrates the difference in the existing system VS the commit reveal system. 

For the ease of illustration, assume `conceal_period = floor(commit_reveal_weight_interval / 360)`, we also suggest users to set the `commit_reveal_weight_interval` to be a multiple of 360 blocks (a tempo).

#### === Existing system ===
| Epoch  (360 blocks = 1 tempo apart)    | Actor |Actions                                                                                         |
|-----------|--------|-----------------------------------------------------------------------------------------|
| n     | Validators |Does evaluation and set weight to the chain.                                          |
| n     | Weight copier | Weights would be available on the chain, weight copier can copy weights available. |
| n | Chain |Calculates consensus based on validator weights.                                           |
| n | Weight copier|Consensus would be available on the chain, weight copier can copy consensus available.   |

#### === Commit reveal system ===
| Epoch (360 blocks = 1 tempo apart)              | Actor | Action                                                                                                                       |
|--------------------|-----------------|---------------------------------------------------------------------------------------------------------|
| n - `conceal_period` | Validators | Does evaluation and set hashed weight hash(hotkey, weight_old) to the chain.                                      |
| n                  | Validators | Set weight to the chain that corresponds to hash(hotkey, weight_old).                                             |
| n                  | Weight copier | Weights would be available on the chain, weight copier can copy weights available and set hash(hotkey, weight_old) to chain. |
| n                  | Chain |Calculates consensus based on hashes received on n - `conceal_period`.                                               |
| n                  | Weight copier | Consensus would be available on the chain, weight copier can copy consensus available and set them as hash(hotkey, weight_old) to chain. |

* notice how when weight copier set weights, the weight it set is already `concel_period` apart from when the weight was generated. 


## Disanostic

### Imports & setup

In [1]:
import os 
from multiprocessing import Pool
import torch 
import pandas as pd
from matplotlib.pyplot import figure
import bittensor as bt

import pickle
from experiment_setup import ExperimentSetup
import plotly.express as px


setup = ExperimentSetup(
    netuids = [4],
    start_block = 4509000 - 100 * 360,
    data_points = 100,
    processes = 30,
)

### Download metagraphs

In [ ]:
from download_metagraphs import DownloadMetagraph
DownloadMetagraph(setup = setup).run()

### Run simulation

In [ ]:
from weight_copy_simulation import WeightCopySimulation
WeightCopySimulation(setup = setup).run_simulation()

### Analysis

#### Getting relative dividend rate

With $D$ as dividend; $S$ as stake; $\mathcal Z$ as the set of validators.
We use the relative dividend rate of the copier $j$,
$$G^j = \frac{D^j/S^j}{\underset{i \in \mathcal Z \setminus \{j\}}{\mathrm{median}} \{D^i/S^i\}}$$
to measure the success of the commit-reveal approach. Here, validator dividend is normalized by the corresponding validator stake as dividend is linear in the amount of stake. Further, we use median as the baseline for comparison.

In [4]:
div_losts = {}
yuma_results = {}

for netuid in setup.netuids:
    div_losts[netuid] = {}
    yuma_results[netuid] = {}
    
    for conceal_period in setup.conceal_periods:
        with open(f"{setup.result_path}/yuma_result_netuid{netuid}_conceal{conceal_period}.pkl", 'rb') as handle:
            _yuma_results = pickle.load(handle)

        dividend = [
            (s["validator_reward_normalized"] / s["stake"]).tolist()
            for idx, s in _yuma_results.items()
        ]

        dividend_df = pd.DataFrame(
            dividend,
            columns=[f"v{i}" for i in range(len(dividend[0]) - 1 )] + ["v_bad"],
            index = _yuma_results.keys()
        )
        
        div_last = dividend_df.iloc[-1]
        div_lost = div_last[-1] / div_last[:-1].median()

        div_losts[netuid][conceal_period] = div_lost
        yuma_results[netuid][conceal_period] = _yuma_results

div_losts = pd.DataFrame(div_losts, dtype='float64')# index as conceal periods 

#### Getting similarity of weight across conceal periods 

In [5]:
similarities = {}

for netuid in setup.netuids:
    similarities[netuid] = {}
    
    for conceal_period in setup.conceal_periods:

        if conceal_period == 0:
            similarities[netuid][conceal_period] = 1
            continue

        similarity = torch.load(f"{setup.result_path}/similarity_netuid{netuid}_conceal{conceal_period}.pt")
        similarities[netuid][conceal_period] = similarity.mean().item()

similarities = pd.DataFrame(similarities, dtype='float64')# index as conceal periods 

### Plotting the changes across conceal periods

In [ ]:
fig = px.line(
    div_losts,
    labels={
        "value": "Relative dividend rate (G)".title(),
        "index": "Conceal period (every 360 blocks)",
        "variable": 'Subnet'
    },
    title="Relative Dividend Rate Of Weight Copier",
    width = 1000,
    height = 500,
)
fig.add_hline(y=1, line_width=3, line_dash="dash", line_color="red", annotation_text = "")
fig.update_layout(template='plotly_white')

fig.show()

### Conclusion 
For the conceal period to be effective, you should set a conceal period large enough to produce enough lost in dividend for the weight copier.

| Dividend gain (G) | Effect                                                                                                     |
|------------------|------------------------------------------------------------------------------------------------------------|
| < 1               | Norminator lost the incentive to delegate to weight copier, weight copier earn less validator take.  |
| < 0.82          | Weigh copier lost the incentive to copy weight.                                                      |

If given a conceal period long enough (>15 hours) and the SN still fail to produce enough lost in dividend, it means that there is not enough churn and weight movement in the SN, so the existing weight copiy fix may not work for your SN. Depending on the situation, you may choose to increase competitiveness/ churn in your SN or just leave the weight copier as is. Cause when there is no churn in the SN, there would be no movement in consensus as well, so the weight copier would not be as beneficial. 

Note that when the conceal period was set too long, it would slow down the discovery of new miners, putting them at risk for deregistration. 